In [1]:
import requests

In [2]:
SRC = "https://raw.githubusercontent.com/ESGF/esgf-vocab/refs/heads/EMD/EMD_example/complete_top_model_example.json"

In [3]:
esdoc = requests.get(SRC).json()
DOI = [x['doi'] for x in esdoc["references"]]

### Optional: capture DOI of model components

In [4]:
for comp in esdoc['model_components']:
    DOI += [x['doi'] for x in comp["references"]]

In [39]:
from copy import deepcopy

### Deduplicate references

In [ ]:
def doi2author(doi):
    res = requests.get(f"https://api.crossref.org/works/{doi}")
    return [[aut['given'], aut['family']] for aut in res.json()["message"]["author"]]


def disambig_list(lst):
    x = lst[0]
    idx = 1
    outlst = []
    while idx < len(lst):
        if not (lst[idx][0] == x[0] and lst[idx][1] == '.'):
            outlst.append(x)
            x = lst[idx]
        idx += 1
    outlst.append(x)
    return outlst

def disambiguate(authors):
    table = {}
    outlst = []
    for a in authors:
        if a[1] in table:
            table[a[1]].append(a[0])
        else:
            table[a[1]] = [a[0]]
    for k in table:
        table[k].sort(reverse=True)
        orglst = deepcopy(table[k])
        res = disambig_list(table[k])  
        # print(f"{k} :  {orglst} {res}"
        for gn in res:
            outlst.append([k, gn])
        
    return outlst

In [5]:
DOI = list(set(DOI))

In [7]:
lists_of_authors = [doi2author(doi) for doi in DOI]  # create a list of lists

In [8]:
authors = list([x for y in lists_of_authors for x in y])

### Run a disambiguation

In [47]:
authlst = disambiguate(authors)

In [1]:
datadoiex = "https://citation.doi.org/metadata?doi=10.22033/ESGF/CMIP6.4700"

In [4]:
res = requests.get(datadoiex).json()

In [8]:
res["contributor"][0]

{'family': 'Fladrich', 'given': 'Uwe'}

In [49]:
import json

In [50]:
datacite = json.load(open("examples/datacite2.json"))

In [51]:
citeeg = json.load(open("examples/cite-eg.json"))

In [53]:
citekeys = list(citeeg.keys())

In [58]:
citekeys.remove("resourceType") # identifier, resourceType

In [59]:
resjson = { key : datacite['data'][0]['attributes'][key]  for key in citekeys }

In [17]:
resjson["identifier"] = { "type" : "DOI" , "id" : datacite['data'][0]['id'] }

In [56]:
citekeys

['creators',
 'titles',
 'publisher',
 'publicationYear',
 'subjects',
 'contributors',
 'dates',
 'language',
 'resourceType',
 'formats',
 'rightsList',
 'descriptions',
 'fundingReferences',
 'relatedIdentifiers']

In [61]:
with open("examples/outputexample.json", "w") as f:
    json.dump(resjson, f, indent=2)